In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests
import re

In [2]:
#leer tablas
tabla_contactos=pd.read_excel('contactos_nuevo.xlsx')
tabla_fechas=pd.read_excel('fecha_control.xlsx')
tabla_fechas["fecha de control"]=pd.to_datetime(tabla_fechas["fecha de control"])
#tabla_error=pd.read_excel('contactos_error.xlsx')


In [3]:
#juntar con la tabla_fechas
tabla_aux=tabla_contactos.merge(tabla_fechas, on='programa', how='left')
tabla_aux["fecha de control"]=pd.to_datetime(tabla_aux["fecha de control"])
tabla_aux=tabla_aux.sort_values('fecha de control')
#contactos unicos
lista_tablas=[]
tabla_contacto_unico=tabla_aux.drop_duplicates(subset=['contacto'])

tabla_principal=tabla_aux.drop_duplicates(subset=['contacto'])

while(True):
    index_A=set(list(tabla_aux['id']))#todo
    print(len(index_A))
    index_P=set(list(tabla_principal['id']))#menos dobles
    index_dif=list(index_A-index_P)
    print(len(index_dif))
    if(len(index_dif)==0):
        break
    else:
        
        tabla_aux=tabla_aux.loc[index_dif]#tabla dif
        tabla_principal=tabla_aux.drop_duplicates(subset=['contacto'])
        tabla_p=tabla_principal.drop(['id','tipo'],axis=1)
        lista_tablas.append(tabla_p)
       
    
j=1
for i in lista_tablas:
    tabla_contacto_unico=tabla_contacto_unico.merge(i, on='contacto', how='left')
    nombre1="programa"+str(j)
    nombre2="flag_exclusion"+str(j)
    nombre3="flag_formal"+str(j)
    nombre4="flag_control"+str(j)
    nombre5="fecha_control"+str(j)
    
    tabla_contacto_unico=tabla_contacto_unico.rename(columns={'programa_x':nombre1,'flag_exclusion_x':nombre2,'flag_formal_x':nombre3,'flag_control_x':nombre4,'fecha de control_x':nombre5})
    j=j+1
    nombre1="programa"+str(j)
    nombre2="flag_exclusion"+str(j)
    nombre3="flag_formal"+str(j)
    nombre4="flag_control"+str(j)
    nombre5="fecha_control"+str(j)
    tabla_contacto_unico=tabla_contacto_unico.rename(columns={'programa_y':nombre1,'flag_exclusion_y':nombre2,'flag_formal_y':nombre3,'flag_control_y':nombre4,'fecha de control_y':nombre5})
                                                 
tabla_contacto_unico.to_excel('tabla_contacto_unico'+".xlsx",header=1,encoding='utf-8')




62198
1818
1818
154
154
4
4
0


In [4]:
#nuevo tablon segun ricardo en base al anterior
tabla_Aux=tabla_contacto_unico
programas=[]
controles=[]

flag_formal_f=[]
fecha_control_f=[]
n=len(tabla_Aux.columns)//5
for i in range(1,n+1):
    nombre="programa"+str(i)
    control="flag_control"+str(i)
    formal="flag_formal"+str(i)
    fecha="fecha_control"+str(i)
    programas.append(list(tabla_Aux[nombre]))
    controles.append(list(tabla_Aux[control]))
    flag_formal_f.append(list(tabla_Aux[formal]))
    fecha_control_f.append(list(tabla_Aux[fecha]))

l=len(programas[0])
nro_programas=[0]*l
nom_programas=[""]*l
ctrl_programas=[""]*l

flag_ctrl=[0]*l
programa_f=[""]*l
formal_f=[0]*l
fecha_f=[""]*l

k=0
for i in programas:
    j=0
    for h in i:
        if(len(str(h))>3):
            if nro_programas[j]==0:
                nom_programas[j]=str(h)
                ctrl_programas[j]=str(controles[k][j])[0]                

            else:
                nom_programas[j]=nom_programas[j]+","+str(h)
                ctrl_programas[j]=ctrl_programas[j]+","+str(controles[k][j])[0]
            nro_programas[j]=nro_programas[j]+1         
                
        j=j+1
    k=k+1
    
    
for i in range(0,len(ctrl_programas)):
    lc=ctrl_programas[i].split(',')
    
    if('1' in ctrl_programas[i]):
        
        pos=lc.index('1')
        flag_ctrl[i]=1
        
    else:
        pos=0
        flag_ctrl[i]=0
    
    programa_f[i]=programas[pos][i]
    formal_f[i]=flag_formal_f[pos][i]
    fecha_f[i]=fecha_control_f[pos][i]
   
    
tabla_contactos_resumen=tabla_Aux[['id','contacto','tipo']]           
tabla_contactos_resumen["nro_programas"]=nro_programas
tabla_contactos_resumen["programas"]=nom_programas
tabla_contactos_resumen["control_programas"]=ctrl_programas
tabla_contactos_resumen["flag_control"]=flag_ctrl
tabla_contactos_resumen["programa_control"]=programa_f
tabla_contactos_resumen["flag_formal"]=formal_f
tabla_contactos_resumen["fecha_control"]=fecha_f

tabla_contactos_resumen.to_excel('tabla_contactos_resumen'+".xlsx",header=1,encoding='utf-8')    


In [5]:
#cruzar con rucs
tabla_rucs=pd.read_excel('MATCH_PAD_CONTRI_CU_MAGNA (2).xlsx')

#tabla_contactos_unicos=pd.read_excel('tablon_contactos_todo.xlsx')
tabla_final=tabla_rucs[['num_ruc','fec_insc','fec_react','fec_alta','regimen','flag_afectacion']]
tabla_final=tabla_final.drop_duplicates(subset=['num_ruc'])
#tabla_final

In [6]:
#join de Rucs unicos con Contactos

tv1=tabla_contactos_resumen["tipo"]!="dni"
tabla_resumen_v1=tabla_contactos_resumen[tv1]
tabla_resumen_v1=tabla_resumen_v1.drop(['id','tipo'],axis=1)

tv2=tabla_contactos_resumen["tipo"]=="dni"
tabla_resumen_v2=tabla_contactos_resumen[tv2]
tabla_resumen_v2= tabla_resumen_v2.drop(['id','tipo'],axis=1)
i=0
for j in range(1,5):
    tabla_final['contacto']=tabla_rucs['telefono'+str(j)]
    tabla_final=tabla_final.merge(tabla_resumen_v1, on='contacto', how='left')
    tabla_final=tabla_final.rename(columns={'contacto':'telefono'+str(j),'nro_programas_x':'nro_programas'+str(i),'programas_x':'programas'+str(i),'control_programas_x':'control_programas'+str(i),'flag_control_x':'flag_control'+str(i),'programa_control_x':'programa_control'+str(i),'flag_formal_x':'flag_formal'+str(i),'fecha_control_x':'fecha_control'+str(i)})
    i=i+1
    tabla_final=tabla_final.rename(columns={'nro_programas_y':'nro_programas'+str(i),'programas_y':'programas'+str(i),'control_programas_y':'control_programas'+str(i),'flag_control_y':'flag_control'+str(i),'programa_control_y':'programa_control'+str(i),'flag_formal_y':'flag_formal'+str(i),'fecha_control_y':'fecha_control'+str(i)})

for j in range(1,3):
    tabla_final['contacto']=tabla_rucs['correo'+str(j)]
    tabla_final=tabla_final.merge(tabla_resumen_v1, on='contacto', how='left')
    tabla_final=tabla_final.rename(columns={'contacto':'correo'+str(j),'nro_programas_x':'nro_programas'+str(i),'programas_x':'programas'+str(i),'control_programas_x':'control_programas'+str(i),'flag_control_x':'flag_control'+str(i),'programa_control_x':'programa_control'+str(i),'flag_formal_x':'flag_formal'+str(i),'fecha_control_x':'fecha_control'+str(i)})
    i=i+1
    tabla_final=tabla_final.rename(columns={'nro_programas_y':'nro_programas'+str(i),'programas_y':'programas'+str(i),'control_programas_y':'control_programas'+str(i),'flag_control_y':'flag_control'+str(i),'programa_control_y':'programa_control'+str(i),'flag_formal_y':'flag_formal'+str(i),'fecha_control_y':'fecha_control'+str(i)})

tabla_final['contacto']=tabla_rucs['dni']
tabla_final=tabla_final.merge(tabla_resumen_v2, on='contacto', how='left')
tabla_final=tabla_final.rename(columns={'contacto':'dni','nro_programas_x':'nro_programas'+str(i),'programas_x':'programas'+str(i),'control_programas_x':'control_programas'+str(i),'flag_control_x':'flag_control'+str(i),'programa_control_x':'programa_control'+str(i),'flag_formal_x':'flag_formal'+str(i),'fecha_control_x':'fecha_control'+str(i)})
i=i+1
tabla_final=tabla_final.rename(columns={'nro_programas_y':'nro_programas'+str(i),'programas_y':'programas'+str(i),'control_programas_y':'control_programas'+str(i),'flag_control_y':'flag_control'+str(i),'programa_control_y':'programa_control'+str(i),'flag_formal_y':'flag_formal'+str(i),'fecha_control_y':'fecha_control'+str(i)})

    
tabla_final.to_excel('tablon_final'+".xlsx",header=1,encoding='utf-8')

In [7]:
#flag nuevos inscritos y reactivados

ind_reactivado=[]

fec_react=list(pd.to_datetime(tabla_final["fec_react"]))
fec_insc=list(pd.to_datetime(tabla_final["fec_insc"]))

for i in range(0,len(fec_react)):
    if(fec_react[i]>fec_insc[i]):
        ind_reactivado.append(1)
    else:
        ind_reactivado.append(0)

flag_nuevo_inscrito=[0]*len(ind_reactivado)
flag_reactivado=[0]*len(ind_reactivado)
for i in range(0,len(fec_react)):
    f=pd.to_datetime("2020-05-23 00:00:00")
    if(fec_insc[i]>=f and ind_reactivado[i]==0):
        flag_nuevo_inscrito[i]=1
    elif(fec_insc[i]>=f and ind_reactivado[i]==1):
         flag_reactivado[i]=1
        
    
tabla_indicadores=tabla_final
tabla_indicadores["ind_reactivado"]=ind_reactivado
tabla_indicadores["flag_nuevo_inscrito"]=flag_nuevo_inscrito
tabla_indicadores["flag_reactivado"]=flag_reactivado
tabla_indicadores.to_excel('tablon_indicadores'+".xlsx",header=1,encoding='utf-8')